<a href="https://colab.research.google.com/github/R12942159/NTU_ML/blob/Hw1/polynomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2023 ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Training config: mini-batch=512, optimizer=Adam, learning rate=0.1 (TODO: Change the config!)



#### v1: 2nd polynomial
#### v2: 0<y<=28.5(remove outliner)
#### v3: Change epoch to 1000 and adding more importance variable by RandomForestClassifier
#### v4: Change lr to 1e*4
#### v5: lower batch size to 8 only

# **Import Some Packages**

In [ ]:
import numpy as np
import csv
import math
import pandas as pd

In [ ]:
import os
os.getcwd()

'/content'

# **Fix random seed**


This is for the reproduction of your result. **DO NOT modify this secton!**


In [ ]:
## Edit: use np.random.seed(seed) (2022.10.12)
seed = 9487
np.random.seed(seed)

# **Download training data**


In [ ]:
!gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"

## Edit (2022.10.14) test.csv: 7th(WS_HR) and last column(pm2.5) is correctly switched.
!gdown --id '155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2' --output "test.csv"

# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
#!gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy
To: /content/train.csv
100% 324k/324k [00:00<00:00, 81.4MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2
To: /content/test.csv
100% 49.0k/49.0k [00:00<00:00, 63.0MB/s]


In [ ]:
def valid(x, y):
  # TODO: Try to filter out extreme values.
  #  ex: If PM2.5 > 100, then we don't use the data to train (return False), otherwise return True,

  return True


# Create your dataset
def parse2train(data, feats):

  x = []
  y = []

  # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
  total_length = data.shape[1] - 8

  for i in range(total_length):
    x_tmp = data[feats, i:i+8] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc.
    y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5

    # Filter out extreme values to train.
    if valid(x_tmp, y_tmp):
      x.append(x_tmp.reshape(-1,))
      y.append(y_tmp)

  # x.shape: (n, 15, 8)
  # y.shape: (n, 1)
  x = np.array(x)
  y = np.array(y)

  return x,y


#**Adam**
* This is our gradient descent algorithm. Adam was implemented.
* You can implement another algorithm such as SGD, which may (or may not) boost the performance.
* However, **modules like sklearn and pytorch are not allowed**.
* Ref: https://arxiv.org/pdf/1412.6980.pdf

![](https://i.imgur.com/jRaebdf.png)



In [ ]:
# TODO: Implement 2-nd polynomial regression version for the report.
def minibatch(x, y, config):

    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]

    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)

    # 2-nd polynomial regression(w1,w2,b)
    w1 = np.full(x[0].shape, 0.1).reshape(-1, 1)
    w2 = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1

    m_t1 = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t2 = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_3 = 0.0

    v_t1 = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t2 = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t_3 = 0.0

    t = 0
    epsilon = 1e-8

    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)

            # Prediction of linear regression
            pred = np.dot(x_batch**2, w1) + np.dot(x_batch, w2) + bias

            # loss
            loss = y_batch - pred

            # Compute gradient
            g_t1 = np.dot((x_batch**2).transpose(), loss) * (-2)
            g_t2 = np.dot(x_batch.transpose(),loss) * (-2)
            g_t_3 = loss.sum(axis=0) * (-2)

            m_t1 = beta_1*m_t1 + (1-beta_1)*g_t1
            m_t2 = beta_1*m_t2 + (1-beta_1)*g_t2
            m_t_3 = 0.9*m_t_3 + (1-0.9)*g_t_3

            v_t1 = beta_2*v_t1 + (1-beta_2)*np.multiply(g_t1, g_t1)
            v_t2 = beta_2*v_t2 + (1-beta_2)*np.multiply(g_t2, g_t2)
            v_t_3 = 0.99*v_t_3 + (1-0.99)*(g_t_3*g_t_3)

            m_cap1 = m_t1/(1-(beta_1**t))
            m_cap2 = m_t2/(1-(beta_1**t))
            m_cap_3 = m_t_3/(1-(0.9**t))

            v_cap1 = v_t1/(1-(beta_2**t))
            v_cap2 = v_t2/(1-(beta_2**t))
            v_cap_3 = v_t_3/(1-(0.99**t))

            w_1 = np.copy(w1)
            w_2 = np.copy(w2)

            # Update weight & bias
            w1 -= ((lr*m_cap1)/(np.sqrt(v_cap1)+epsilon))
            w2 -= ((lr*m_cap2)/(np.sqrt(v_cap2)+epsilon))
            bias -= (lr*m_cap_3)/(math.sqrt(v_cap_3)+epsilon)


    return w1, w2, bias

In [ ]:
from argparse import Namespace

# TODO: Tune the config to boost your performance.
train_config = Namespace(
    batch_size = 512,
    lr = 1e-1,
    lam = 0.001,
    epoch = 1,
)


# **Training your regression model**

In [ ]:
data = pd.read_csv("/content/train.csv")
data

,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
0,10.8,0.32,1.7,8.6,10.3,22.9,21,0.6,0.0,71,1.9,172,171,0.6,15
1,10.8,0.27,1.6,6.2,7.8,23.8,20,1.4,0.0,71,1.7,161,129,1.8,13
2,11.0,0.25,0.9,5.4,6.3,27.4,21,0.8,0.0,68,1.6,152,147,1.5,12
3,11.0,0.23,0.7,3.1,3.8,29.5,21,1.8,0.0,68,1.6,138,145,1.7,9
4,11.3,0.22,0.8,2.9,3.8,30.7,16,1.9,0.0,67,1.6,140,139,1.7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,29.0,0.41,1.2,14.0,15.3,23.0,21,1.4,0.0,74,2.8,149,168,2.0,14
5770,28.2,0.33,1.7,11.7,13.5,19.5,23,2.1,0.0,78,2.3,187,179,2.5,15
5771,28.0,0.29,1.3,9.1,10.4,17.6,17,1.5,0.0,78,2.0,173,200,1.5,13
5772,28.0,0.27,1.4,9.5,11.0,15.4,17,1.1,0.0,75,1.8,171,135,0.9,10


In [ ]:
# Choose your features to train.
# Hint:
# 1. You can select more than one feature.
# 2. You should select "good" features.

# TODO: Carefully justify which feature should be chosen.
feats = [2]

In [ ]:
# Training data preprocessing.

data = data.values
train_data = np.transpose(np.array(np.float64(data)))
train_x, train_y = parse2train(train_data, feats)

In [ ]:
# Train your regression model

w1, w2, bias = minibatch(train_x, train_y, train_config)
print(w1.shape, w2.shape, bias.shape)

(8, 1) (8, 1) (1,)


In [ ]:
w1, w2, bias

(array([[ 0.0324149 ],
        [-0.10955948],
        [-0.01614332],
        [ 0.09518239],
        [-0.051995  ],
        [ 0.0658696 ],
        [-0.07257947],
        [ 0.08801479]]),
 array([[0.38487713],
        [0.27877768],
        [0.26185372],
        [0.24145826],
        [0.22826823],
        [0.29885901],
        [0.26746609],
        [0.36222515]]),
 array([0.79817754]))

# **Testing:**


In [ ]:
def parse2test(data, feats):
  x = []
  for i in range(90):
    x_tmp = data[feats,8*i: 8*i+8]
    x.append(x_tmp.reshape(-1,))

  # x.shape: (n, 15, 8)
  x = np.array(x)
  return x


In [ ]:
data = pd.read_csv('test.csv')
print(data)
data = data.values

test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)

     AMB_TEMP    CO   NO   NO2   NOx    O3  PM10  WS_HR  RAINFALL    RH  SO2  \
0        27.5  0.22  0.7   9.0   9.8  13.2  31.0    1.2       0.0  79.0  1.7   
1        27.2  0.17  0.4   5.0   5.4  15.7  20.0    1.5       0.0  79.0  1.6   
2        26.8  0.17  0.4   4.3   4.8  12.8  16.0    1.6       0.0  81.0  1.3   
3        26.7  0.19  0.4   4.1   4.5  12.0  21.0    1.7       0.0  80.0  1.5   
4        26.4  0.22  0.4   4.1   4.6  10.1  23.0    2.2       0.0  81.0  1.5   
..        ...   ...  ...   ...   ...   ...   ...    ...       ...   ...  ...   
715      16.0  0.26  0.3   3.9   4.2  47.1  34.0    2.7       0.0  70.0  0.5   
716      15.6  0.25  0.4   3.3   3.7  44.1  27.0    3.0       0.0  74.0  0.6   
717      15.7  0.24  0.4   3.7   4.1  44.1  29.0    2.9       0.0  73.0  0.6   
718      15.1  0.24  0.6  10.5  11.1  29.9   9.0    0.8       0.0  95.0  0.6   
719      15.8  0.28  0.6   6.0   6.7  40.5  28.0    3.0       0.0  74.0  0.5   

     WD_HR  WIND_DIREC  WIND_SPEED  PM2

# **Write result as .csv**

---



In [ ]:
 with open('r12942159_2nd_poly_v1.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])

    print(test_x.shape)
    for i in range(int(test_x.shape[0])):
      # Prediction of linear regression
      # prediction = (np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
      # Prediction of 2nd polynomial regression
      prediction = (np.dot(np.reshape(w1,-1),test_x[i]) + np.dot(np.reshape(w2,-1),test_x[i]) + bias)[0]

      writer.writerow([i, prediction])

(90, 8)
